https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

# K means

- unsupervised
- hard clustering
- non-parametric 

## advantage

- scalable
- flexible
- simple
- efficient

## disadvantage.

- assumues shperical clusters 

- non - convex loss function
  - have to try multiple initialization
  - non-local split-and-merge
    - merge two nearby clusters and split a big cluster into two
   
- sensitive to the initializaton
  - different initialization output differently
 
  -  have to try multiple initialization and pick the best one.

## assumption / inductive bias 

- check if the data
  - Isotropic
    - multiple well-separated clusters in 2D space.
  - Anisotropic
    - different variance
    - in different direction.

- members of the same cluster are close / similar to each other
  - close in distance 
- members of different clusters are not similar
 - far in distance

- spherical clusters

- all of the clusters are the same shape

- should have the same variance. 


-  assume data lives in a euclidean space??
-  points are in a high-dimensional??????


## metrics/ loss function

- Uk(1<=k<=K) centroids
- r(nk) - assingment
  - in {0,1}
  - shows if n the data is in class k

### objective function

- J = $\sum^{N}_{n=1}\sum^{K}_{k=1}r(nk)|| x_n - Uk||^2_2$
  - non-convex (no guaranteed to convere to the global minimum)
  - 
 
### Silhouette analysis

- s(i) = $\frac{b(i)-a(i)}{max(a(i),b(i))}$
  - a(i) - mean distance to points in its own cluster
  - b(i) - mean distance to points in nearest neighboring cluster
  - is in [-1,1]
- s(i) => 1 optimal clustering
- s(i) => 0 the point lies between clusters
- s(i) =>-1 the point is missclassified. 


## hyper parameter 

- the number of class - k
  - 

- distance
  - Euclidean
  - Cosine
  - Jaccard
  - edit distance



## data preprocessing before 

- scaling because algorithm uses distance.


## algorithm 

- initialize the cluster centers
  - randomly
  - kmeans ++
  - naive sharding
- assign all the data to group based on the distance between cluster centers and each point
- recalculate the cluster centers
  - mean
- check if the model have to update cluster centers
  - converge or not.
 
**Pesudo code**

- initialze k centroids
- repeat until convergence
  - fix Uk and minimize based on r(nk)
  - fix r(nk) and minimize based on Uk
  - recalculate centroids
    - mean

## initializations 

### K-Means ++ 

- choose centroids one by one.
  - first, choose one uniformly at random
    - choose subsequent centroid
    - get the distance from each point to the each centroids
      - D(x) is the closest distance. 
    - choose next centroids with probability proportional to D(x)^2
      - $p(x) = D(x)^2 / sum(D(x)^2)$
    - normalize probabilies before selection. 
    

### Percentile-based initialization

In [ ]:
from math import sqrt, floor
import numpy as np
def random(ds, k, random_state=123):
    np.random.seed(random_state=123):
    
centroids = []
m = np.shape(ds)[0]
for _ in range(k):
r = np.random.randint(0, m-1)
centroids.append(ds[r])
return np.array(centroids)
def plus_plus(ds, k, random_state=42):
"""
    Create cluster centroids using the k-means++ algorithm.
    Parameters
    ----------
    ds : numpy array
        The dataset to be used for centroid initialization.
    k : int
        The desired number of clusters for which centroids are required.
    Returns
    -------
    centroids : numpy array
        Collection of k centroids as a numpy array.
    Inspiration from here: https://stackoverflow.com/questions/5466323/how-could-one-implement-the-k-means-algorithm
    """
np.random.seed(random_state)
centroids = [ds[0]]
for _ in range(1, k):
dist_sq = np.array([min([np.inner(c-x,c-x) for c in centroids]) for x in ds])
probs = dist_sq/dist_sq.sum()
cumulative_probs = probs.cumsum()
r = np.random.rand()
for j, p in enumerate(cumulative_probs):
if r < p:
i = j
break
centroids.append(ds[i])
return np.array(centroids)
def naive_sharding(ds, k):
"""
    Create cluster centroids using deterministic naive sharding algorithm.
    
    Parameters
    ----------
    ds : numpy array
        The dataset to be used for centroid initialization.
    k : int
        The desired number of clusters for which centroids are required.
    Returns
    -------
    centroids : numpy array
        Collection of k centroids as a numpy array.
    """
def _get_mean(sums, step):
"""Vectorizable ufunc for getting means of summed shard columns."""
return sums/step
n = np.shape(ds)[1]
m = np.shape(ds)[0]
centroids = np.zeros((k, n))
composite = np.mat(np.sum(ds, axis=1))
ds = np.append(composite.T, ds, axis=1)
ds.sort(axis=0)
step = floor(m/k)
vfunc = np.vectorize(_get_mean)
for j in range(k):
if j == k-1:
centroids[j:] = vfunc(np.sum(ds[j*step:,1:], axis=0), step)
else:
centroids[j:] = vfunc(np.sum(ds[j*step:(j+1)*step,1:], axis=0), step)
return centroids
